# Hyperparameter Optimization Techinique

### First Model Creation 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

C:\Users\rushi\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\rushi\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\rushi\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


#### load dataset

In [3]:
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df['Glucose'] = np.where(df['Glucose']==0,df['Glucose'].median(),df['Glucose'])
df['Insulin'] = np.where(df['Insulin']==0,df['Insulin'].median(),df['Insulin'])
df['SkinThickness'] = np.where(df['SkinThickness']==0,df['SkinThickness'].median(),df['SkinThickness'])
df['BloodPressure'] = np.where(df['BloodPressure']==0,df['BloodPressure'].median(),df['BloodPressure'])
df['BMI'] = np.where(df['BMI']==0,df['BMI'].median(),df['BMI'])
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Class
0,6,148.0,72.0,35.0,30.5,33.6,0.627,50,1
1,1,85.0,66.0,29.0,30.5,26.6,0.351,31,0
2,8,183.0,64.0,23.0,30.5,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [5]:
x=df.drop('Class',axis=1)
y=df['Class']

In [6]:
x.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,72.0,35.0,30.5,33.6,0.627,50
1,1,85.0,66.0,29.0,30.5,26.6,0.351,31
2,8,183.0,64.0,23.0,30.5,23.3,0.672,32
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33


In [7]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Class, dtype: int64

##### Perform train test split

In [8]:
from sklearn.model_selection import train_test_split,GridSearchCV
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,random_state=21)

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf_cls = RandomForestClassifier(n_estimators=10).fit(x_train,y_train)
prediction = rf_cls.predict(x_test)

In [10]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[86  8]
 [36 24]]
0.7142857142857143
              precision    recall  f1-score   support

           0       0.70      0.91      0.80        94
           1       0.75      0.40      0.52        60

    accuracy                           0.71       154
   macro avg       0.73      0.66      0.66       154
weighted avg       0.72      0.71      0.69       154



#### The main parameter used by a RandomForestClassifier are :

- Criterion = used to evaluate quality of split
- max_depth = maximum number of levels allowed in each tree
- max_featured = maximum number of features considered when splitting node
- min_samples_leaf = minimum samples which  can be stored in a tree leaf
- min_samples_split = minimun samples necessary in a node to cause node splitting
- n_estimators = number of trees in ensemble

### -  Manual Hyperparameter Tuning
- selecting based on what i want without caring of model

In [11]:
rf_cls = RandomForestClassifier(n_estimators=200,criterion='gini',max_depth=6,
                                max_features=5,min_samples_leaf=3).fit(x_train,y_train)
prediction = rf_cls.predict(x_test)
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[85  9]
 [30 30]]
0.7467532467532467
              precision    recall  f1-score   support

           0       0.74      0.90      0.81        94
           1       0.77      0.50      0.61        60

    accuracy                           0.75       154
   macro avg       0.75      0.70      0.71       154
weighted avg       0.75      0.75      0.73       154



## Now, we use Hyperparameter tuning Methods 

##### (A,B,C,D)
- 1. GridSearchCV  -----> it check every area randomly(C,A,D,B)
- 2. RandomizedSearchCV -----> it predict some area (B,C)

for better result we use RandomizedSearchCV first then we apply GridSearchCV

### 1. RandomizedSearchCV

In [12]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(200,2000,10)]
max_features = ['auto','sqrt','log2']
max_depth = [int(x) for x in np.linspace(10,1000,10)]
min_samples_split = [1,2,3,4,5,6,7,8,9,10]
min_samples_leaf = [1,2,3,4,5,6,7,8,9,10]

In [13]:
#Create random Grid
random_grid = {'n_estimators':n_estimators,
              'max_features':max_features,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf,
              'criterion':['entropy','gini']
              }
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'criterion': ['entropy', 'gini']}


In [14]:
rf = RandomForestClassifier()
rf_randomCV = RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                                random_state=100,n_jobs=-1)
rf_randomCV.fit(x_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.7min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10],
                                        'min_samples_split': [1, 2, 3, 4, 5, 6,
                                                              7, 8, 9, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
   

In [15]:
rf_randomCV.best_params_

{'n_estimators': 200,
 'min_samples_split': 7,
 'min_samples_leaf': 3,
 'max_features': 'log2',
 'max_depth': 450,
 'criterion': 'entropy'}

In [16]:
rf_randomCV_grid = rf_randomCV.best_estimator_

In [17]:
rf_randomCV_grid

RandomForestClassifier(criterion='entropy', max_depth=450, max_features='log2',
                       min_samples_leaf=3, min_samples_split=7,
                       n_estimators=200)

In [18]:
prediction = rf_randomCV_grid.predict(x_test)
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[85  9]
 [28 32]]
0.7597402597402597
              precision    recall  f1-score   support

           0       0.75      0.90      0.82        94
           1       0.78      0.53      0.63        60

    accuracy                           0.76       154
   macro avg       0.77      0.72      0.73       154
weighted avg       0.76      0.76      0.75       154



###### Before Hyperparameter tuning accuracy is "0.7402" and after Hyperparameter tuning accuracy is "0.7532"

### 2. Now, apply GridSearchCV

In [19]:
rf_randomCV.best_params_

{'n_estimators': 200,
 'min_samples_split': 7,
 'min_samples_leaf': 3,
 'max_features': 'log2',
 'max_depth': 450,
 'criterion': 'entropy'}

In [20]:
param_grid = {
    'criterion':[rf_randomCV.best_params_['criterion']],
    'max_depth':[rf_randomCV.best_params_['max_depth']],
    'max_features':[rf_randomCV.best_params_['max_features']],
    'min_samples_leaf':[rf_randomCV.best_params_['min_samples_leaf'],
                       rf_randomCV.best_params_['min_samples_leaf']+2,
                       rf_randomCV.best_params_['min_samples_leaf']+4],
    'min_samples_split':[rf_randomCV.best_params_['min_samples_split']-2,
                        rf_randomCV.best_params_['min_samples_split']-1,
                        rf_randomCV.best_params_['min_samples_split'],
                        rf_randomCV.best_params_['min_samples_split']+1,
                        rf_randomCV.best_params_['min_samples_split']+2],
    'n_estimators':[rf_randomCV.best_params_['n_estimators']-200,
                   rf_randomCV.best_params_['n_estimators']-100,
                   rf_randomCV.best_params_['n_estimators'],
                   rf_randomCV.best_params_['n_estimators']+100,
                   rf_randomCV.best_params_['n_estimators']+200],
}
print(param_grid)

{'criterion': ['entropy'], 'max_depth': [450], 'max_features': ['log2'], 'min_samples_leaf': [3, 5, 7], 'min_samples_split': [5, 6, 7, 8, 9], 'n_estimators': [0, 100, 200, 300, 400]}



#### Fit GridSearchCV to data

In [21]:
rf =RandomForestClassifier()
grid_search = GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(x_train,y_train)

Fitting 10 folds for each of 75 candidates, totalling 750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 304 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-1)]: Done 507 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:  1.7min finished


GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [450],
                         'max_features': ['log2'],
                         'min_samples_leaf': [3, 5, 7],
                         'min_samples_split': [5, 6, 7, 8, 9],
                         'n_estimators': [0, 100, 200, 300, 400]},
             verbose=2)

In [22]:
rf_grid = grid_search.best_estimator_

In [23]:
rf_grid

RandomForestClassifier(criterion='entropy', max_depth=450, max_features='log2',
                       min_samples_leaf=3, min_samples_split=7,
                       n_estimators=300)

In [24]:
prediction = rf_grid.predict(x_test)
print(confusion_matrix(y_test,prediction))
print(accuracy_score(y_test,prediction))
print(classification_report(y_test,prediction))

[[84 10]
 [28 32]]
0.7532467532467533
              precision    recall  f1-score   support

           0       0.75      0.89      0.82        94
           1       0.76      0.53      0.63        60

    accuracy                           0.75       154
   macro avg       0.76      0.71      0.72       154
weighted avg       0.75      0.75      0.74       154



## Result

#### Before Hyperparameter tuning accuracy is "0.71"
#### Manual Hyperparameter tuning accuracy is "0.74" 
#### RandomSearchCV tuning accuracy is "0.77" 
#### GribSearchCV accuracy is "0.77"

# Automated Hyperparameter Tuning
- Bayesian Optimization
- Gradient Descent
- Evolutionary Algorithms



In [25]:
from hyperopt import hp,tpe,STATUS_OK,fmin,Trials

In [27]:
#Domain Scpace 
space = {
    'criterion':hp.choice('criterion',['entropy','gini']),
    'max_depth':hp.quniform('max_depth',10,1200,10),
    'max_feature':hp.choice('max_features',['auto','sqrt','log2',None]),
    'min_samples_leaf':hp.uniform('min_samples_leaf',0,0.5),
    'min_samples_split':hp.uniform('min_samples_split',0,1),
    'n_estimators':hp.choice('n_estimators',[10,50,300,530,750,1000,1200])    
}

In [38]:
def objactive(space):
    model = RandomForestClassifier(criterion=space['criterion'],
                                  max_depth=space['max_depth'],
                                  max_features=space['max_feature'],
                                  min_samples_leaf=space['min_samples_leaf'],
                                  min_samples_split=space['min_samples_split'],
                                  n_estimators=space['n_estimators']
                                  )
    accuracy = cross_val_score(model,x_train,y_train,cv=5).mean()
    
    return {'loss': -accuracy , 'status': STATUS_OK }

In [39]:
from sklearn.model_selection import cross_val_score
trials = Trials()
best = fmin(fn=objactive,
           space=space,
           algo=tpe.suggest,
           max_evals=70,
           trials=trials)
best

100%|████████████████████████████████████████████████| 70/70 [04:33<00:00,  3.91s/trial, best loss: -0.775223244035719]


{'criterion': 1,
 'max_depth': 160.0,
 'max_features': 2,
 'min_samples_leaf': 0.07850630203553924,
 'min_samples_split': 0.2933917422110708,
 'n_estimators': 2}

In [40]:
crit = {0:'entropy',1:'gini'}
feat = {0:'auto',1:'sqrt',2:'log2',3:None}
est = {0:10,1:50,2:300,3:530,4:750,5:1000,6:1200}
print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
log2
300


In [41]:
best['min_samples_leaf']

0.07850630203553924

In [54]:
train_rf = RandomForestClassifier(criterion=crit[best['criterion']],
                                max_depth=best['max_depth'],
                                max_features=feat[best['max_features']],
                                min_samples_leaf=best['min_samples_leaf'],
                                min_samples_split=best['min_samples_split'],
                                n_estimators=est[best['n_estimators']]                               
                                )
train_rf.fit(x_train,y_train)
pred_rf = train_rf.predict(x_test)

In [55]:
print(confusion_matrix(y_test,pred_rf))
print(accuracy_score(y_test,pred_rf))
print(classification_report(y_test,pred_rf))

[[85  9]
 [42 18]]
0.6688311688311688
              precision    recall  f1-score   support

           0       0.67      0.90      0.77        94
           1       0.67      0.30      0.41        60

    accuracy                           0.67       154
   macro avg       0.67      0.60      0.59       154
weighted avg       0.67      0.67      0.63       154



## Genetic Algoritham TPOT

In [56]:
from tpot import TPOTClassifier

In [57]:
n_estimators = [int(x) for x in np.linspace(200,2000,10)]
max_features = ['auto','sqrt','log2']
max_depth = [int(x) for x in np.linspace(10,1000,10)]
min_samples_split = [1,2,3,4,5,6,7,8,9,10]
min_samples_leaf = [1,2,3,4,5,6,7,8,9,10]

#Create random Grid
tpot_grid = {'n_estimators':n_estimators,
              'max_features':max_features,
              'max_depth':max_depth,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf,
              'criterion':['entropy','gini']
              }
print(tpot_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'criterion': ['entropy', 'gini']}


In [58]:
tpot_class = TPOTClassifier(generations=5,population_size=24,offspring_size=12,
                           verbosity=2,early_stop=12,
                           config_dict={'sklearn.ensemble.RandomForestClassifier':tpot_grid},
                           cv=4,scoring='accuracy')
tpot_class.fit(x_train,y_train)

Version 0.11.5 of tpot is outdated. Version 0.11.6 was released Monday October 26, 2020.



Generation 1 - Current best internal CV score: 0.7882819794584501
Generation 2 - Current best internal CV score: 0.7882819794584501
Generation 3 - Current best internal CV score: 0.7899265766912826
Generation 4 - Current best internal CV score: 0.7899265766912826
Generation 5 - Current best internal CV score: 0.7899477973007385
Best pipeline: RandomForestClassifier(input_matrix, criterion=gini, max_depth=890, max_features=sqrt, min_samples_leaf=3, min_samples_split=4, n_estimators=800)


TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                 

In [59]:
accuracy = tpot_class.score(x_test,y_test)
print(accuracy)

0.7597402597402597


In [62]:
tpot_class

TPOTClassifier(config_dict={'sklearn.ensemble.RandomForestClassifier': {'criterion': ['entropy',
                                                                                      'gini'],
                                                                        'max_depth': [10,
                                                                                      120,
                                                                                      230,
                                                                                      340,
                                                                                      450,
                                                                                      560,
                                                                                      670,
                                                                                      780,
                                                                                 

In [64]:
accuracy = tpot_class.score(x_test,y_test)
print(accuracy)

0.7597402597402597


## Optimize Hyper Parameter of the model using Optuna
- 1. create objective function (apply HPT)
- 2. 

In [76]:
import optuna

ModuleNotFoundError: No module named 'optuna'